In [58]:
import os
import requests
import json
import time
import base64

for env_name in ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY']:
    os.environ.pop(env_name, None)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def call_storydiffusion():
    url = 'http://127.0.0.1:8977/story_storydiffusion'


    character_prompts = '[1] a man\n[2] a woman'
    prompt_array = '[1] running on a beach\n[1] eat rice\n[2] play games'
    char_files = ''
    payload1 = {
        "sd_type": 'Unstable',
        "model_type": 'Only Using Textual Description',
        "upload_images": [],
        "num_steps": 20,
        "style_name": 'Japanese Anime',
        "Ip_Adapter_Strength": 0.5,
        "style_strength_ratio": 20,
        "guidance_scale": 5,
        "seed_": 1,
        "sa32_": 0.3,
        "sa64_": 0.5,
        "id_length_": 1,
        "general_prompt": character_prompts,
        "negative_prompt": 'bad anatomy, bad hands, missing fingers, extra fingers, three hands, three legs, bad arms, missing legs, missing arms, poorly drawn face, bad face, fused face, cloned face, three crus, fused feet, fused thigh, extra crus, ugly fingers, horn, cartoon, cg, 3d, unreal, animate, amputation, disconnected limbs',
        "prompt_array": prompt_array,
        "G_height": 768,
        "G_width": 768,
        "comic_type": 'Classic Comic Style',
        "font_choice": 'Inkfree.ttf',
        "char_files": char_files,
        "response_format": 'save_root'
    }

    # character_prompts = ''
    prompt_array = '[1] read a book in a dim room, sit on chair\n[2] hold a long sword, angry\n[2] lay down on yellow grass \n[1] riding on horse, smiles'
    char_files = ['/gpfs/public/vl/wangjiazhi/00_projs/StoryDiffusion/results/20241114-151937/weights/[1].pt', 
                  '/gpfs/public/vl/wangjiazhi/00_projs/StoryDiffusion/results/20241114-151937/weights/[2].pt'
                ]
    char_files = '\n'.join(char_files)
    payload2 = {
        "sd_type": 'Unstable',
        "model_type": 'Only Using Textual Description',
        "upload_images": [],
        "num_steps": 20,
        "style_name": 'Japanese Anime',
        "Ip_Adapter_Strength": 0.5,
        "style_strength_ratio": 20,
        "guidance_scale": 5,
        "seed_": 1,
        "sa32_": 0.3,
        "sa64_": 0.5,
        "id_length_": 1,
        "general_prompt": character_prompts,
        "negative_prompt": 'bad anatomy, bad hands, missing fingers, extra fingers, three hands, three legs, bad arms, missing legs, missing arms, poorly drawn face, bad face, fused face, cloned face, three crus, fused feet, fused thigh, extra crus, ugly fingers, horn, cartoon, cg, 3d, unreal, animate, amputation, disconnected limbs',
        "prompt_array": prompt_array,
        "G_height": 768,
        "G_width": 768,
        "comic_type": 'Classic Comic Style',
        "font_choice": 'Inkfree.ttf',
        "char_files": char_files,
        "response_format": 'save_root'
    }

    response = requests.post(url, data=json.dumps(payload2))  # Send a POST request to the FastAPI backend
    return response


t0 = time.time()

resp = call_storydiffusion()

t1 = time.time()
print(f"Request took {t1 - t0:.2f} seconds")

# Check if the request was successful
if resp.status_code == 200:
    # Parse the JSON response content
    try:
        data = resp.json()  # This converts the response to a Python dictionary
        # print(data)
    except json.JSONDecodeError:
        print("Response content is not in JSON format.")
else:
    print(f"Request failed with status code {resp.status_code}")
    print(resp.content)

print(f"is success: {data['success']}")
print(data['data'])

Request took 16.21 seconds
is success: True
/gpfs/public/vl/wangjiazhi/00_projs/StoryDiffusion/results/20241114-152801


In [60]:
import re

def replace_second_and_subsequent_characters(text, char_dict):
    # Find all occurrences of '[character_*]' and store their match positions
    matches = list(re.finditer(r'\[character_(\d+)\]', text))
    
    # Convert matches to a list to modify later occurrences
    for i, match in enumerate(matches[1:], start=1):  # Start with the second match
        character_num = match.group(1)  # Extract the character number, e.g., '1' or '5'
        if character_num in char_dict:
            # Replace the occurrence in the text with the description from the dictionary
            text = text[:match.start()] + char_dict[character_num] + text[match.end():]
            # Update the remaining matches to account for text length change
            for j in range(i + 1, len(matches)):
                matches[j] = re.search(r'\[character_(\d+)\]', text, pos=matches[j].start() + len(char_dict[character_num]) - len(match.group(0)))
                
    return text

# Example usage:
text = "[character_1] tightly hugs [character_6]'s leg, gazing up despairingly. Barren wilderness, clear sky, dusty ground."
char_dict = {"1": "desc about the character", "6": "7xx"}

updated_text = replace_second_and_subsequent_characters(text, char_dict)
print(updated_text)

[character_1] tightly hugs 7xx's leg, gazing up despairingly. Barren wilderness, clear sky, dusty ground.
